# Observables

This example shows how to declare various Observables, how to evaluate them and how to extract and display their values.

## Setup the environment

In [1]:
import at
import sys
import numpy as np
if sys.version_info.minor < 9:
    from importlib_resources import files, as_file
else:
    from importlib.resources import files, as_file

In [2]:
from at import Observable, ObservableList, OrbitObservable, GlobalOpticsObservable, LocalOpticsObservable
from at import MatrixObservable, TrajectoryObservable, EmittanceObservable, LatticeObservable, GeometryObservable

## Load a test lattice

In [3]:
fname = 'hmba.mat'
with as_file(files('machine_data') / fname) as path:
    hmba_lattice = at.load_lattice(path)

## Create Observables

Create an empty ObservableList:

In [4]:
obs1=ObservableList(hmba_lattice)

Horizontal closed orbit on all Monitors:

In [5]:
obs1.append(OrbitObservable(at.Monitor, axis='x'))

Create a 2{sup}`nd` ObservableList:

In [6]:
obs2=ObservableList(hmba_lattice)

Vertical $\beta$ at all monitors, with a target and bounds.

The vertical $\beta$ is constrained in the interval
[*target*+*low_bound* *target*+*up_bound*], so here [*-Infinity 7.0*]

The residual will be zero within the interval.

In [7]:
obs2.append(LocalOpticsObservable(at.Monitor, 'beta', plane=1, target=7.0, bounds=(-np.inf, 0.0)))

check the concatenation of ObservableLists:

In [8]:
allobs = obs1 + obs2

Full transfer matrix to `BPM02`:

In [9]:
allobs.append(MatrixObservable("BPM_02"))

Maximum of vertical beta on monitors:

In [10]:
allobs.append(LocalOpticsObservable(at.Monitor, 'beta', plane='v', statfun=np.amax))

First 4 coordinates of the closed orbit at Quadrupoles:

In [11]:
allobs.append(LocalOpticsObservable(at.Quadrupole, 'closed_orbit', plane=slice(4), target=0.0, weight=1.e-6))

Position along the lattice of all quadrupoles:

In [12]:
allobs.append(LocalOpticsObservable(at.Quadrupole, 's_pos'))

Phase advance between elements 33 and 101 in all planes.

First define a custom evaluation function:

In [13]:
def phase_advance(ring, elemdata):
    mu = elemdata.mu
    return (mu[-1] - mu[0])

Then create the constraints.

The evaluation should return one value per refpoint (2 here). Alternatively, it may return a single
value, but then one must set `summary=True`.

In [14]:
allobs.append(LocalOpticsObservable([33, 101], phase_advance, use_integer=True, summary=True))

Horizontal tune with the integer part:

In [15]:
allobs.append(GlobalOpticsObservable('tune', plane=0, use_integer=True))

Total phase advance at the end of the lattice (all planes):

In [16]:
allobs.append(LocalOpticsObservable(at.End, 'mu', use_integer=True))

Chromaticity in all planes:

In [17]:
allobs.append(GlobalOpticsObservable('chromaticity', plane=None))

Average of sextupole strengths:

In [18]:
allobs.append(LatticeObservable(at.Sextupole, 'H', statfun=np.mean))

Average of sextupole strengths:

In [19]:
allobs.append(LatticeObservable(at.Sextupole, 'PolynomB', index=2))

Horizontal emittance:

In [20]:
allobs.append(EmittanceObservable('emittances', plane='x'))

Ring circumference:

In [21]:
def circumference(ring):
    return ring.get_s_pos(len(ring))[0]
allobs.append(Observable(circumference))

p{sub}`x` component of the trajectory on all monitors:

In [22]:
allobs.append(TrajectoryObservable(at.Monitor,axis='px'))

In [24]:
allobs.append(GeometryObservable(at.Monitor, 'x'))

## Evaluation

An input trajectory is required for the trajectory Observable

In [25]:
r_in = np.zeros(6)
r_in[0] = 0.001
r_in[2] = 0.001
allobs.evaluate(hmba_lattice.enable_6d(copy=True), r_in=r_in, dp=0.0, initial=True)

Extract a single Observable value (phase advance between elements 3 and 101):

In [26]:
allobs[6].value

array([9.38969042e+00, 2.99742405e+00, 1.40412855e-14])

Get the list of all Observable values:

In [27]:
allobs.values

[array([-3.02189723e-09,  4.50695010e-07,  4.08205708e-07,  2.37899777e-08,
        -1.31783789e-08,  2.47230566e-08, -2.95310962e-08, -4.05598220e-07,
        -4.47398212e-07, -2.24850930e-09]),
 array([5.30279703, 7.17604152, 6.55087808, 2.31448878, 3.40498445,
        3.405044  , 2.3146451 , 6.55106241, 7.17614175, 5.30283837]),
 array([[[-1.08194106e+00,  3.18809568e+00,  0.00000000e+00,
           0.00000000e+00,  8.22407787e-02, -1.72158979e-05],
         [-6.80522735e-01,  1.08099571e+00,  0.00000000e+00,
           0.00000000e+00,  4.90131193e-02, -1.02601760e-05],
         [ 0.00000000e+00,  0.00000000e+00,  7.55929650e-01,
           3.87059271e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -6.79279293e-01,
          -2.15524755e+00,  0.00000000e+00,  0.00000000e+00],
         [-1.13309009e-08, -1.08615600e-07,  0.00000000e+00,
           0.00000000e+00,  9.99995907e-01, -2.09334442e-04],
         [ 2.93742206e-03,  6.73567963e-02,  0.0000

Get a pretty output of all Observables.

As no variation was made, *Actual* values are always equal to *Initial* values.

The residual is zero for all Observables for which no *target* was specified

In [ ]:
print(allobs)